In [ ]:
import numpy as np
import cv2
import os
import json
import re

In [ ]:
dataset_dir = "../midv_dataset/"

## remove all the card whose quadrialteral is out of the image

In [58]:
useable_images = []
for currentpath, folders, files in os.walk(dataset_dir):
    for file in files:
#         if file != "TA42_22.tif":
#             continue
        each_filepath = os.path.join(currentpath, file)
        if len(re.findall(r"\/\w{2}\/\w{2}\d{2}\_\d{2}\.tif$", each_filepath))<1:
            continue
        
        img = cv2.imread(each_filepath)
        img_h, img_w = img.shape[:2]
        json_path = re.sub(r"\.tif$",".json",each_filepath)
        json_path = re.sub("\/images\/",'/ground_truth/',json_path, re.DOTALL)
        

        with open(json_path,"r") as f:
            quad_json = json.load(f)
        quad_points = np.array(quad_json["quad"])
        
        if ((quad_points[:,1] > img_h).any() or (quad_points[:,1] < 0).any()) or \
            ((quad_points[:,0] > img_w).any() or (quad_points[:,0] < 0).any()):
#             print(each_filepath)
            continue
            
        useable_images.append(each_filepath)
        txt_path = re.sub("\.json$", ".txt", json_path)
        with open(txt_path,"w") as f:
            out_txt = "0 " + " ".join(str(x[0]) + " " + str(x[1]) for x in quad_points)
            f.write(out_txt)
        
        
        

In [64]:

with open("../trainable_image.txt","w") as f:
    out_str = "\n".join(x.replace("../","./") for x in useable_images)
    f.write(out_str)

## Generate txt labels from *trainable_images.txt* file

In [65]:
trainable_img_file_path = "../trainable_image.txt"
with open(trainable_img_file_path) as f:
    trainable_img = f.read().split("\n")
    
for img_path in trainable_img:
    json_path = re.sub("\/images\/",'/ground_truth/',img_path, re.DOTALL)
    with open(json_path,"r") as f:
        quad_json = json.load(f)
    quad_points = np.array(quad_json["quad"])
    txt_path = re.sub("\.json$", ".txt", json_path)
    with open(txt_path,"w") as f:
        out_txt = "0 " + " ".join(str(x[0]) + " " + str(x[1]) for x in quad_points)
        f.write(out_txt)

# train test split

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
trainable_img_file_path = "../trainable_dataset_files/trainable_image.txt"
with open(trainable_img_file_path) as f:
    trainable_img = f.read().split("\n")

In [ ]:
x_train, x_test = train_test_split(trainable_img, test_size=0.15, random_state=32)

In [12]:
len(x_train),len(x_test)

(8322, 1469)

In [14]:
with open("../trainable_dataset_files/train.part", "w") as f:
    out_text = "\n".join(x_train)
    f.write(out_text)
with open("../trainable_dataset_files/test.part", "w") as f:
    out_text = "\n".join(x_test)
    f.write(out_text)

In [ ]:
x_train